# Exploring data

When working with a dataset, the first step is usually to understand what data and metadata it contains.
In this chapter we explore how scipp supports this.

This tutorial contains exercises, but solutions are included directly.
We encourage you to download this notebook and run through it step by step before looking at the solutions.
We recommend to use a recent version of *JupyterLab*:
The solutions are included as hidden cells and shown only on demand.

First, in addition to importing `scipp`, we import `scippneutron` since this is required for loading Nexus files:

In [ ]:
import scipp as sc
import scippneutron as scn

We  start by loading some data, in this case measured with a prototype of the [LoKI](https://europeanspallationsource.se/instruments/loki) detectors at the [LARMOR beamline](https://www.isis.stfc.ac.uk/Pages/Larmor.aspx):

In [ ]:
data = scn.data.tutorial_dense_data()

In practice, we would use [scippneutron.load](https://scipp.github.io/scippneutron/generated/scippneutron.load.html#scippneutron.load) or [scippneutron.load_nexus](https://scipp.github.io/scippneutron/generated/scippneutron.load_nexus.html#scippneutron.load_nexus) to load the data from a NeXus file, but the tutorial data comes bundled with scippneutron to make it easily available.
See [Tutorial and Test Data](../developer/getting-started.rst#tutorial-and-test-data) for a way to customize where the data is stored.

Note that the exercises in the following are fictional and do not represent the actual SANS data reduction workflow.

## Step 1: Use the HTML representation to see what the loaded data contains

The HTML representation is what Jupyter displays for a scipp object.

- Take some time to explore this view and try to understand all the information (dimensions, dtypes, units, ...).
- Note that sections can be expanded, and values can shown by clicking the icons to the right.

In [ ]:
data

Whenever you get stuck with one of the exercises below we recommend consulting the HTML representations of the objects you are working with.

## Step 2: Plot the data

Scipp objects (variables, data arrays, or datasets) can be plotted using the `plot()` method.
Alternatively `sc.plot(obj)` can be used, e.g., when `obj` is a Python `dict` of scipp data arrays.
Since this is neutron-scattering data, we can also use the "instrument view", provided by `scn.instrument_view(obj)` (assuming `scippneutron` was imported as `scn`).

### Exercise

- Plot the loaded data and familiarize yourself with the controls.
- Create the instrument view and familiarize yourself with the controls.

### Solution

In [ ]:
data.plot()

In [ ]:
scn.instrument_view(data)

## Step 3: Exploring meta data

Above we saw that many attributes are scalar variables with `dtype=DataArray`.
The single value in a scalar variable is accessed using the `value` property.
Compare:

In [ ]:
data.attrs['proton_charge_by_period']

In [ ]:
data.attrs['proton_charge_by_period'].value

### Exercise

1. Find some attributes of `data` with `dtype=DataArray` and plot their `value`.
   Also try `sc.table(attr.value)` to show a table representation (where `attr` is an attribute of your choice).
2. Find and plot a monitor.
3. Try to normalize `data` to monitor 1.
   Why does this fail?
4. Plot all the monitors on the same plot.
   Note that `sc.plot()` can be used with a Python `dict` for this purpose: `sc.plot({'a':something, 'b':else})`.
5. Convert all the monitors from `'tof'` to `'wavelength'` using, e.g., `mon1_wav = scn.convert(mon1, 'tof', 'wavelength', scatter=False)`.
6. Inspect the HTML view and note how the "unit conversion" changed the dimensions and units.
7. Re-plot all the monitors on the same plot, now in `'wavelength'`.

### Solution

In [ ]:
sc.table(data.attrs['DCMagField2'].value)

In [ ]:
try:
    data / data.attrs['monitor1'].value
except sc.CoordError:
    print(
        """Data and monitor are in unit TOF, but pixels and monitors
are at different position, so data is not comparable."""
    )

A monitor can be converted to wavelength as follows:

In [ ]:
mon1 = data.attrs['monitor1'].value
scn.convert(mon1, 'tof', 'wavelength', scatter=False)

In [ ]:
monitors = {f'monitor{i}': data.attrs[f'monitor{i}'].value for i in [1, 2, 3, 4, 5]}
sc.plot(monitors, norm='log')

In [ ]:
converted_monitors = {
    f'monitor{i}': scn.convert(
        data.attrs[f'monitor{i}'].value, 'tof', 'wavelength', scatter=False
    )
    for i in [1, 2, 3, 4, 5]
}
sc.plot(converted_monitors, norm='log')

## Step 4: Fixing metadata

### Exercise

Consider the following (hypothetical) problems with the metadata stored in `data`:

1. The `sample_position` coord (`data.coords['sample_position']`) is wrong, shift the sample by `delta = sc.vector(value=np.array([0.01,0.01,0.04]), unit='m')`.
2. Because of a glitch in the timing system the time-of-flight has an offset of $2.3~\mu s$.
   Fix the corresponding coordinate.
3. Use the HTML view of `data` to verify that you applied the corrections/calibrations there, rather than in a copy.

### Solution

In [ ]:
data.coords['sample_position'] += sc.vector(value=[0.01, 0.01, 0.04], unit='m')
data.coords['tof'] += 2.3 * sc.Unit('us')  # note how we forgot to fix the monitor's TOF
data

Note how adding such offsets fails if we fail to specify a unit:

In [ ]:
try:
    data.coords['tof'] += 2.3
except sc.UnitError as e:
    print(e)

This has several advantages:
- We are protected from accidential errors.
  If someone changes the unit of data or metatdata without our knowledge, e.g., from `us` to `ms` this mechanism protects us from silent errors corrupting the data.
- It makes the code clearer and more readable, both for others as well as for our future selves.

The offset to the sample could also be done component-wise using the special `fields` property of variables with "vector" dtype, e.g.,

In [ ]:
data.coords['sample_position'].fields.z += 0.001 * sc.Unit('m')

## Step 5: A closer look at the data

The 2-D plot we obtain above by default is often not very enlightening.
Define:

In [ ]:
counts = data.sum('tof')

### Exercise

1. Create a plot of `counts` and also try the instrument view.
2. How many counts are there in total, in all spectra combined?
3. Plot a single spectrum of `data` as a 1-D plot using the slicing syntax `array[dim_name, integer_index]` to access the spectrum.

### Solution

In [ ]:
# slice is optional, making plot more readable in the documentation
counts['spectrum', 56000:62000].plot()

In [ ]:
scn.instrument_view(counts, norm='log')

In [ ]:
# counts.sum('spectrum') # would be another solution
data.sum().value

In [ ]:
data['spectrum', 10000].plot()